In [ ]:
import numpy as np
import sys
import os
import plotly.graph_objects as go
import plotly.express as px
import io
import PIL
sys.path.append('../Functions/py_functions/') # This path is so that within each function file, you can import the other function files with relative paths
sys.path.append('../') # This path is so that we can import the functions folder from the root directory compared to where this file is
from Functions.py_functions.constants import *
from Functions.py_functions.car_configuration import Car
from Functions.py_functions.gps_importer import *
from Functions.py_functions.mmd import MMD
from Functions.py_functions.tire_model.tire_model_utils import * # this has the tire models
from Functions.py_functions.steady_state_solver.ls_optimize import LS_Solver
from Functions.py_functions.steady_state_solver.iterative import Iterative_Solver
from Functions.py_functions.steady_state_solver.parachute import Parachute

In [ ]:
car = Car()
car.k_c = 1400 * FTLB_TO_NM
car.k_f = 374
car.k_r = 928
car.z_f = 0.13 * IN_TO_M
car.z_r = 1.0 * IN_TO_M
car.set_lltd()
car.set_tire(H_R20_18X6_7)

solver = Iterative_Solver()
# solver = LS_Solver()
# solver = Parachute()

In [ ]:
fig = go.Figure()
fig.update_xaxes(title_text='Lat Acc (G)')
fig.update_yaxes(title_text='Cn')

vels = np.linspace(6, 25, 25)#[5.0, 7.5, 10.0, 12.5, 15.0, 17.5, 20.0, 22.5, 25.0, 27.5, 30.0]
obj_steps = [(0, 0)] * len(vels)

figs = []
# find max and min of all data
max_cn, max_ay = 15, 2
for i, v in enumerate(vels):
    mmd = MMD(car, solver=solver)
    print(f"Running {v:.1f} m/s")
    mmd.mmd_sweep(v, size=21, seeded=True, max_beta=25, max_delta=25, mu=0.65)
    # mmd.clear_high_sa(max_sa=15)
    fig2 = go.Figure()
    fig2.update_xaxes(title_text='Lat Acc (G)')
    fig2.update_yaxes(title_text='Cn')
    obj_range = mmd.add_mmd(fig, f"{v:.1f} m/s")
    figs.append(PIL.Image.open(io.BytesIO(mmd.plot_mmd(pub=True, return_fig=True).to_image(format="png"))))
    obj_steps[i] = obj_range
    max_cn = max(max_cn, np.max(mmd.cn))
    max_ay = max(max_ay, np.max(mmd.ay)/G)

for ob in fig.data[obj_steps[0][0]:obj_steps[0][1]]: ob.visible = True
steps = []
for i, v in enumerate(vels):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)}],  # layout attribute
        label=f"{v:.1f} m/s"
    )
    step["args"][0]["visible"][obj_steps[i][0]:obj_steps[i][1]] = [True] * (obj_steps[i][1] - obj_steps[i][0])
    steps.append(step)

sliders = [dict(active=10, currentvalue={"prefix": "Velocity: "}, pad={"t": 50}, steps=steps)]

fig.update_layout(sliders=sliders, title_text=f"2D MMD Across Velocity", height=1024, width=1024) # , template="plotly_dark"
fig.update_xaxes(range=[-max_ay*1.1, max_ay*1.1])
fig.update_yaxes(range=[-max_cn*1.1, max_cn*1.1])

# create animated GIF
figs[0].save(
        "test.gif",
        save_all=True,
        append_images=figs[1:],
        optimize=True,
        duration=500,
        loop=0,
    )

fig.show()